In [55]:
import os
import pandas as pd
import nibabel as nib
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from skimage.measure import block_reduce
import numpy as np
from concurrent.futures import ProcessPoolExecutor
import time
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorly as tl

#Debugging import
import importlib
var = 'TensorDecisionTreeRegressorP' #the published version of code
package = importlib.import_module(var)
for name, value in package.__dict__.items():
    if not name.startswith("__"):
        globals()[name] = value

from TensorDecisionTreeRegressorP import *

import os
import nibabel as nib
import numpy as np
import matplotlib as plt
import pandas as pd
from sklearn.model_selection import train_test_split


# File path to the CSV file
csv_file = '/Users/zc56/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/ADNIData.csv'
df = pd.read_csv(csv_file)

# Remove rows where ADAS11_bl is missing (NaN)
df_cleaned = df.dropna(subset=['ADAS11_bl'])

# Extract the 'ADAS11_bl' column as the y variable
y_variable = df_cleaned['ADAS11_bl'].values

# Split the dataframe based on the DX_bl column values
cn_group = df_cleaned[df_cleaned['DX_bl'] == 'CN']
ad_group = df_cleaned[df_cleaned['DX_bl'] == 'AD']
lmci_group = df_cleaned[df_cleaned['DX_bl'] == 'LMCI']

# Display the counts for each group after removing NA
print(f"CN group size: {cn_group.shape[0]}")
print(f"AD group size: {ad_group.shape[0]}")
print(f"LMCI group size: {lmci_group.shape[0]}")

# Directory containing the 3D images
directory = '/Users/zc56/Documents/CommenDesktop/RICE/MyProject/Bayes_Tensor_Tree/3D-images/3D-Images/bl'

# Initialize dictionaries to hold the images and y values for each group
cn_images, ad_images, lmci_images = [], [], []
cn_y, ad_y, lmci_y = [], [], []

# Function to load the images based on PTID matching and append y values
def load_images_and_y(group, image_list, y_list):
    for _, row in group.iterrows():
        ptid = row['PTID']
        # Find the corresponding file based on PTID
        filename = f'{ptid}.nii.gz'
        file_path = os.path.join(directory, filename)
        
        if os.path.exists(file_path):
            # Load the NIfTI file
            img = nib.load(file_path)
            data = img.get_fdata()
            
            # Append the 3D image data and y value to the respective lists
            image_list.append(data)
            y_list.append(row['ADAS11_bl'])
        else:
            print(f"File {filename} not found.")

# Load images and y values for each group
load_images_and_y(cn_group, cn_images, cn_y)
load_images_and_y(ad_group, ad_images, ad_y)
load_images_and_y(lmci_group, lmci_images, lmci_y)

# Convert lists of 3D images and y values to NumPy arrays
if cn_images:
    cn_tensor = np.stack(cn_images, axis=0)
    cn_y = np.array(cn_y)
    print(f"CN 4D tensor shape: {cn_tensor.shape}")
    print(f"CN y shape: {cn_y.shape}")
else:
    print("No CN images loaded.")

if ad_images:
    ad_tensor = np.stack(ad_images, axis=0)
    ad_y = np.array(ad_y)
    print(f"AD 4D tensor shape: {ad_tensor.shape}")
    print(f"AD y shape: {ad_y.shape}")
else:
    print("No AD images loaded.")

if lmci_images:
    lmci_tensor = np.stack(lmci_images, axis=0)
    lmci_y = np.array(lmci_y)
    print(f"LMCI 4D tensor shape: {lmci_tensor.shape}")
    print(f"LMCI y shape: {lmci_y.shape}")
else:
    print("No LMCI images loaded.")


CN group size: 229
AD group size: 187
LMCI group size: 401
CN 4D tensor shape: (229, 48, 48, 48)
CN y shape: (229,)
AD 4D tensor shape: (187, 48, 48, 48)
AD y shape: (187,)
LMCI 4D tensor shape: (401, 48, 48, 48)
LMCI y shape: (401,)


coarsen with mean function, CP criterion

In [56]:
X_train, X_test, y_train, y_test = train_test_split(lmci_tensor, lmci_y, test_size=0.2, random_state=42)


print(X_train.shape,y_train.shape)
model  =  TensorDecisionTreeRegressor(max_depth=2, min_samples_split=12,split_method='lowrank_LS', split_rank=4, CP_reg_rank=12, Tucker_reg_rank=12, n_mode=3)
model.use_mean_as_threshold  =  False
model.sample_rate  =  .5
X_coarsen_shape = (1,4,4,4)
X_coarsen_func = np.mean
X_train_c = block_reduce(X_train,block_size=X_coarsen_shape, func=X_coarsen_func)
X_test_c = block_reduce(X_test,block_size=X_coarsen_shape, func=X_coarsen_func)
#middle_z = X_train_c.shape[2] // 2
#X_train_c = X_train_c[:,:,:,middle_z:middle_z+2]
#X_test_c = X_test_c[:,:,:,middle_z:middle_z+2]
X_train_c = X_train_c + np.random.random(X_train_c.shape) * 1e-3
X_test_c = X_test_c + np.random.random(X_test_c.shape) * 1e-3
print(X_train_c.shape,y_train.shape,X_test_c.shape)
model.fit(X_train_c,y_train)
#model.prune(X_val=None, y_val=None, model_type='mean', alpha=0.0000000000001)


predictions = model.predict(X_train_c,regression_method='mean')
print(f"mean train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='cp')
print(f"CP train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='tucker')
print(f"Tucker train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train)) 

predictions = model.predict(X_test_c,regression_method='mean')
print(f"mean test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='cp')
print(f"CP test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='tucker')
print(f"Tucker test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))

(320, 48, 48, 48) (320,)
(320, 12, 12, 12) (320,) (81, 12, 12, 12)
None-->X (13, 12, 12, 12) [[[[4.27883286e-04 8.73819947e-04 7.34819252e-04 ... 2.24367504e-04
    1.73486489e-04 7.64462016e-04]
   [1.08773686e-05 4.09381994e-04 3.48828885e-04 ... 1.37231047e-04
    3.17545735e-04 1.74788606e-04]
   [2.98628477e-04 1.77897812e-04 9.83593928e-04 ... 7.40431421e-05
    7.45999924e-05 9.32839315e-04]
   ...
   [3.99206539e-04 5.45304974e-04 3.97384441e-04 ... 7.13793715e-04
    6.39672030e-04 3.13278553e-04]
   [6.63114828e-04 7.64145107e-04 2.12513556e-04 ... 9.95315195e-04
    8.98156751e-04 1.99123841e-04]
   [2.51778677e-04 7.79584848e-04 4.61438565e-04 ... 9.69030836e-04
    4.51844796e-04 8.23426231e-04]]

  [[2.07241662e-04 9.00443223e-04 3.15272684e-04 ... 6.22757170e-04
    9.83399774e-05 1.47198679e-04]
   [4.56134715e-04 4.81119303e-04 2.74504991e-04 ... 3.53258122e-04
    1.50960029e-04 9.12026702e-04]
   [1.02939087e-04 3.51726851e-04 4.17810905e-04 ... 2.24976656e-05
    9.

In [41]:
X_train, X_test, y_train, y_test = train_test_split(lmci_tensor, lmci_y, test_size=0.2, random_state=42)


print(X_train.shape,y_train.shape)
model  =  TensorDecisionTreeRegressor(max_depth=15, min_samples_split=12,split_method='variance_LS', split_rank=4, CP_reg_rank=12, Tucker_reg_rank=12, n_mode=3)
model.use_mean_as_threshold  =  False
model.sample_rate  =  .5
X_coarsen_shape = (1,4,4,4)
X_coarsen_func = np.max
X_train_c = block_reduce(X_train,block_size=X_coarsen_shape, func=X_coarsen_func)
X_test_c = block_reduce(X_test,block_size=X_coarsen_shape, func=X_coarsen_func)
#middle_z = X_train_c.shape[2] // 2
#X_train_c = X_train_c[:,:,:,middle_z:middle_z+2]
#X_test_c = X_test_c[:,:,:,middle_z:middle_z+2]
X_train_c = X_train_c + np.random.random(X_train_c.shape) * 1e-3
X_test_c = X_test_c + np.random.random(X_test_c.shape) * 1e-3
print(X_train_c.shape,y_train.shape,X_test_c.shape)
model.fit(X_train_c,y_train)
model.prune(X_val=None, y_val=None, model_type='mean', alpha=0.0000000000001)


predictions = model.predict(X_train_c,regression_method='mean')
print(f"mean train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='cp')
print(f"CP train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='tucker')
print(f"Tucker train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train)) 

predictions = model.predict(X_test_c,regression_method='mean')
print(f"mean test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='cp')
print(f"CP test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='tucker')
print(f"Tucker test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
model.print_tree()

(320, 48, 48, 48) (320,)
(320, 12, 12, 12) (320,) (81, 12, 12, 12)
mean train RSE:  1.0
CP train RSE:  0.0009892286201279579
Tucker train RSE:  4.8931525081339475e-05
mean test RSE:  1.004552307627379
CP test RSE:  5.676450550911371
Tucker test RSE:  2.851184274995978
  has  0  child nodes, and  320  samples.


Tucker each sample and preserve cores: 

In [53]:
def decompose_tensor(tensor, ranks):
    core, factors = tucker(tensor, rank=ranks)
    return core, factors

for r in range(12, 25):
    ranks = [r, r, r]
    
    core_tensor_list = []  # Collect all core tensors here
    for i, tensor in enumerate(lmci_tensor):
        core, factors = decompose_tensor(tensor, ranks)
        core_tensor_list.append(tl.to_numpy(core))  # Append the core tensor for each tensor
    
    core_tensor_array = np.array(core_tensor_list)
    
    # Ensure core_tensor_array and lmci_y have the same number of samples
    if core_tensor_array.shape[0] != len(lmci_y):
        print(f"Error: Mismatched number of samples between core_tensor_array and lmci_y.")
        continue
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(core_tensor_array, lmci_y, test_size=0.2, random_state=42)
    
    print(f"Rank {r} - X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    
    # Initialize the model
    model = TensorDecisionTreeRegressor(
        max_depth=2, min_samples_split=12, split_method='variance_LS',
        split_rank=4, CP_reg_rank=12, Tucker_reg_rank=12, n_mode=3
    )
    model.use_mean_as_threshold = False
    model.sample_rate = 0.5

    # Add noise to the training and test data
    X_train_c = X_train + np.random.random(X_train.shape) * 1e-3
    X_test_c = X_test + np.random.random(X_test.shape) * 1e-3
    
    print(f"Rank {r} - X_train_c shape: {X_train_c.shape}, X_test_c shape: {X_test_c.shape}")
    
    # Fit the model on the training data
    model.fit(X_train_c, y_train)

    # Predictions and RSE calculations
    predictions = model.predict(X_train_c, regression_method='mean')
    print(f"Rank {r} - mean train RSE: ", np.mean((predictions - y_train)**2) / np.var(y_train))

    predictions = model.predict(X_train_c, regression_method='cp')
    print(f"Rank {r} - CP train RSE: ", np.mean((predictions - y_train)**2) / np.var(y_train))

    predictions = model.predict(X_train_c, regression_method='tucker')
    print(f"Rank {r} - Tucker train RSE: ", np.mean((predictions - y_train)**2) / np.var(y_train))

    # Test set predictions
    predictions = model.predict(X_test_c, regression_method='mean')
    print(f"Rank {r} - mean test RSE: ", np.mean((predictions - y_test)**2) / np.var(y_test))

    predictions = model.predict(X_test_c, regression_method='cp')
    print(f"Rank {r} - CP test RSE: ", np.mean((predictions - y_test)**2) / np.var(y_test))

    predictions = model.predict(X_test_c, regression_method='tucker')
    print(f"Rank {r} - Tucker test RSE: ", np.mean((predictions - y_test)**2) / np.var(y_test))

    # Print the decision tree
    model.print_tree()

    print("-" * 50)








Rank 12 - X_train shape: (320, 12, 12, 12), y_train shape: (320,)
Rank 12 - X_train_c shape: (320, 12, 12, 12), X_test_c shape: (81, 12, 12, 12)
Rank 12 - mean train RSE:  0.8228489612314556
Rank 12 - CP train RSE:  3.3676645002757606e-05
Rank 12 - Tucker train RSE:  1.4713355819631617e-05
Rank 12 - mean test RSE:  1.2576151411847094
Rank 12 - CP test RSE:  4.30481122904766
Rank 12 - Tucker test RSE:  4.05929748362216
  if X[:, 4 , 0 , 2 ] <=  -13.161445345431883
   if X[:, 0 , 3 , 1 ] <=  -17.666859026900557
     has  0  child nodes, and  12  samples.
   else: # if X[:, 0 , 3 , 1 ] >  -17.666859026900557
     has  0  child nodes, and  67  samples.
  else: # if X[:, 4 , 0 , 2 ] >  -13.161445345431883
   if X[:, 4 , 9 , 1 ] <=  0.616641211340333
     has  0  child nodes, and  131  samples.
   else: # if X[:, 4 , 9 , 1 ] >  0.616641211340333
     has  0  child nodes, and  110  samples.
--------------------------------------------------
Rank 13 - X_train shape: (320, 13, 13, 13), y_train 

In [22]:
X_train, X_test, y_train, y_test = train_test_split(lmci_tensor, lmci_y, test_size=0.2, random_state=42)


print(X_train.shape,y_train.shape)
model  =  TensorDecisionTreeRegressor(max_depth=2, min_samples_split=12,split_method='lowrank_LS', split_rank=6, CP_reg_rank=12, Tucker_reg_rank=12, n_mode=3)
model.use_mean_as_threshold  =  False
model.sample_rate  =  .5
X_coarsen_shape = (1,4,4,4)
X_coarsen_func = np.max
X_train_c = block_reduce(X_train,block_size=X_coarsen_shape, func=X_coarsen_func)
X_test_c = block_reduce(X_test,block_size=X_coarsen_shape, func=X_coarsen_func)
#middle_z = X_train_c.shape[2] // 2
#X_train_c = X_train_c[:,:,:,middle_z:middle_z+2]
#X_test_c = X_test_c[:,:,:,middle_z:middle_z+2]
X_train_c = X_train_c + np.random.random(X_train_c.shape) * 1e-3
X_test_c = X_test_c + np.random.random(X_test_c.shape) * 1e-3
print(X_train_c.shape,y_train.shape,X_test_c.shape)
model.fit(X_train_c,y_train)
#model.prune(X_val=None, y_val=None, model_type='mean', alpha=0.0000000000001)


predictions = model.predict(X_train_c,regression_method='mean')
print(f"mean train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='cp')
print(f"CP train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='tucker')
print(f"Tucker train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train)) 

predictions = model.predict(X_test_c,regression_method='mean')
print(f"mean test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='cp')
print(f"CP test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='tucker')
print(f"Tucker test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))

(320, 48, 48, 48) (320,)
(320, 12, 12, 12) (320,) (81, 12, 12, 12)


KeyboardInterrupt: 

In [19]:
model.print_tree()

  if X[:, 7 , 5 , 3 ] <=  4.260466255984082
   if X[:, 8 , 8 , 3 ] <=  9.160833150936087
     has  0  child nodes, and  160  samples.
   else: # if X[:, 8 , 8 , 3 ] >  9.160833150936087
     has  0  child nodes, and  19  samples.
  else: # if X[:, 7 , 5 , 3 ] >  4.260466255984082
   if X[:, 7 , 6 , 5 ] <=  2.045395497052817
     has  0  child nodes, and  127  samples.
   else: # if X[:, 7 , 6 , 5 ] >  2.045395497052817
     has  0  child nodes, and  14  samples.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(lmci_tensor, lmci_y, test_size=0.2, random_state=42)


print(X_train.shape,y_train.shape)
model  =  TensorDecisionTreeRegressor(max_depth=2, min_samples_split=12,split_method='variance_LS', split_rank=4, CP_reg_rank=12, Tucker_reg_rank=12, n_mode=3)
model.use_mean_as_threshold  =  False
model.sample_rate  =  .5
#X_coarsen_shape = (1,4,4,4)
#X_coarsen_func = np.max
#X_train_c = block_reduce(X_train,block_size=X_coarsen_shape, func=X_coarsen_func)
#X_test_c = block_reduce(X_test,block_size=X_coarsen_shape, func=X_coarsen_func)
#middle_z = X_train_c.shape[2] // 2
#X_train_c = X_train_c[:,:,:,middle_z:middle_z+2]
#X_test_c = X_test_c[:,:,:,middle_z:middle_z+2]
X_train = X_train + np.random.random(X_train.shape) * 1e-3
X_test = X_test + np.random.random(X_test.shape) * 1e-3
print(X_train.shape,y_train.shape,X_test.shape)
model.fit(X_train,y_train)
#model.prune(X_val=None, y_val=None, model_type='mean', alpha=0.0000000000001)


predictions = model.predict(X_train,regression_method='mean')
print(f"mean train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train,regression_method='cp')
print(f"CP train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train,regression_method='tucker')
print(f"Tucker train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train)) 

predictions = model.predict(X_test,regression_method='mean')
print(f"mean test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test,regression_method='cp')
print(f"CP test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test,regression_method='tucker')
print(f"Tucker test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
model.print_tree()
#terminated at 1602m 37.6s

In [24]:
X_train, X_test, y_train, y_test = train_test_split(lmci_tensor, lmci_y, test_size=0.2, random_state=42)


print(X_train.shape,y_train.shape)
model  =  TensorDecisionTreeRegressor(max_depth=6, min_samples_split=4,split_method='variance_LS', split_rank=4, CP_reg_rank=12, Tucker_reg_rank=12, n_mode=3)
model.use_mean_as_threshold  =  False
model.sample_rate  =  .5
X_coarsen_shape = (1,4,4,4)
X_coarsen_func = np.max
X_train_c = block_reduce(X_train,block_size=X_coarsen_shape, func=X_coarsen_func)
X_test_c = block_reduce(X_test,block_size=X_coarsen_shape, func=X_coarsen_func)
middle_z = X_train_c.shape[2] // 2
X_train_c = X_train_c[:,:,:,middle_z]
X_test_c = X_test_c[:,:,:,middle_z]


num_train_samples_to_insert = 150  # You can modify this as needed

# Randomly select `num_train_samples_to_insert` indices from the training set
indices_to_insert = np.random.choice(X_train_c.shape[0], size=num_train_samples_to_insert, replace=False)

# Select the samples from training data
X_train_selected = X_train_c[indices_to_insert]
y_train_selected = y_train[indices_to_insert]

# Append the selected training samples to the test set
X_test_c = np.concatenate((X_test_c, X_train_selected), axis=0)
y_test = np.concatenate((y_test, y_train_selected), axis=0)

print(f"New test set size: {X_test_c.shape[0]} samples")






X_train_c = X_train_c+np.ones_like(X_train_c)*1e-3
print(X_train_c.shape,y_train.shape,X_test_c.shape)
model.fit(X_train_c,y_train)

predictions = model.predict(X_train_c,regression_method='mean')
print(f"mean train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='cp')
print(f"CP train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train))
predictions = model.predict(X_train_c,regression_method='tucker')
print(f"Tucker train RSE: ", np.mean((predictions-y_train)**2)/np.var(y_train)) 

predictions = model.predict(X_test_c,regression_method='mean')
print(f"mean test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='cp')
print(f"CP test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))
predictions = model.predict(X_test_c,regression_method='tucker')
print(f"Tucker test RSE: ", np.mean((predictions-y_test)**2)/np.var(y_test))

(320, 48, 48, 48) (320,)
New test set size: 231 samples
(320, 12, 12) (320,) (231, 12, 12)
mean train RSE:  0.3011783208551451
CP train RSE:  0.0010428227718747068
Tucker train RSE:  0.0007032311104089574
mean test RSE:  0.8966147697076805
CP test RSE:  0.8248397915079682
Tucker test RSE:  0.823242470951701


In [15]:
model.print_tree()

  if X[:, 3 , 2 ] <=  3.91728360748291
   if X[:, 0 , 4 ] <=  2.046891284942627
     if X[:, 3 , 0 ] <=  1.666015697479248
         has  0  child nodes, and  3  samples.
     else: # if X[:, 3 , 0 ] >  1.666015697479248
         has  0  child nodes, and  24  samples.
   else: # if X[:, 0 , 4 ] >  2.046891284942627
     if X[:, 1 , 3 ] <=  3.9491985569000243
         has  0  child nodes, and  76  samples.
     else: # if X[:, 1 , 3 ] >  3.9491985569000243
         has  0  child nodes, and  5  samples.
  else: # if X[:, 3 , 2 ] >  3.91728360748291
   if X[:, 3 , 9 ] <=  4.2429953346252445
     if X[:, 3 , 3 ] <=  3.4235966930389403
         has  0  child nodes, and  33  samples.
     else: # if X[:, 3 , 3 ] >  3.4235966930389403
         has  0  child nodes, and  142  samples.
   else: # if X[:, 3 , 9 ] >  4.2429953346252445
     if X[:, 5 , 10 ] <=  1.8898146877288817
         has  0  child nodes, and  1  samples.
     else: # if X[:, 5 , 10 ] >  1.8898146877288817
         has  0  chil

In [18]:
# Gradient Boosting Regressor Class
from GradientBoosting import *
# Initialize Gradient Boosting Regressor with a simple decision tree as weak learner
#weak_learner = TensorDecisionTreeRegressor(max_depth=3, min_samples_split=4, split_method='variance',split_rank=my_rank,n_mode=my_n_mode)
#weak_learner.use_mean_as_threshold = True
#weak_learner.sample_rate = 1.0
weak_learner = model
gradient_boosting_regressor = GradientBoostingRegressor(
    n_estimators=20,
    learning_rate=0.1,
    weak_learner=weak_learner
)
#gradient_boosting_regressor.pruning = True
# Fit a single tree model
weak_learner.fit(X_train_c, y_train)
weak_predictions = weak_learner.predict(X_test_c)


# Fit the Gradient Boosting model
gradient_boosting_regressor.fit(X_train_c, y_train, X_test_c, y_test)

fitting started
0 0 training RMSE === 7.737706822618617
0 0 testing RMSE === 12.70127769621351
0 1 training RMSE === 5.075573656109799
0 1 testing RMSE === 9.451351882185381
0 2 training RMSE === 3.155224187776032
0 2 testing RMSE === 7.14634730297179
0 3 training RMSE === 1.8342950779692009
0 3 testing RMSE === 5.442365625372838
0 4 training RMSE === 0.9642088447604471
0 4 testing RMSE === 4.279619456183524
0 5 training RMSE === 0.4488559618796715
0 5 testing RMSE === 3.4676999658285994
0 6 training RMSE === 0.2067429968578536
0 6 testing RMSE === 2.962657612282183
0 7 training RMSE === 0.16431394873001617
0 7 testing RMSE === 2.6725386413342003
0 8 training RMSE === 0.26821799995970846
0 8 testing RMSE === 2.582447203335448
0 9 training RMSE === 0.4779906249566817
0 9 testing RMSE === 2.568569728851243
0 10 training RMSE === 0.7647256150731122
0 10 testing RMSE === 2.6710408672349097
0 11 training RMSE === 1.1088395439904153
0 11 testing RMSE === 2.8410831334338558
0 12 training RMSE

In [21]:
from GradientBoosting import *

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(lmci_tensor, lmci_y, test_size=0.2, random_state=42)

# Coarsen the data using block_reduce
X_coarsen_shape = (1, 4, 4, 4)
X_coarsen_func = np.max
X_train_c = block_reduce(X_train, block_size=X_coarsen_shape, func=X_coarsen_func)
X_test_c = block_reduce(X_test, block_size=X_coarsen_shape, func=X_coarsen_func)

# Instantiate the random forest with multiple trees
forest_model = TensorRandomForestRegressor(n_estimators=5, max_depth=6, min_samples_split=4, split_method='variance_LS', 
                                           split_rank=4, CP_reg_rank=12, Tucker_reg_rank=12, n_mode=4, sample_rate=0.5)

# Train the random forest on the coarsened data
forest_model.fit(X_train_c, y_train)

# Make predictions on the train data
train_predictions = forest_model.predict(X_train_c, regression_method='mean')
print(f"mean train RSE: ", np.mean((train_predictions - y_train) ** 2) / np.var(y_train))

# Make predictions on the test data
test_predictions = forest_model.predict(X_test_c, regression_method='mean')
print(f"mean test RSE: ", np.mean((test_predictions - y_test) ** 2) / np.var(y_test))

Tree 1/5, Training RMSE (relative to variance): 0.3037996228206997
Tree 2/5, Training RMSE (relative to variance): 0.23710628341168502
Tree 3/5, Training RMSE (relative to variance): 0.27486627609185244
Tree 4/5, Training RMSE (relative to variance): 0.3025809097856107
Tree 5/5, Training RMSE (relative to variance): 0.28804661978948526
mean train RSE:  0.43826742557856746
mean test RSE:  1.325849619738386


In [22]:
from GradientBoosting import *

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(lmci_tensor, lmci_y, test_size=0.2, random_state=42)
X_train = X_train + np.random.uniform(low=-1e-3, high=1e-3, size=X_train.shape)
X_test = X_test + np.random.uniform(low=-1e-3, high=1e-3, size=X_test.shape)
forest_model = TensorRandomForestRegressor(n_estimators=10, max_depth=6, min_samples_split=4, split_method='variance_LS', 
                                           split_rank=4, CP_reg_rank=6, Tucker_reg_rank=6, n_mode=4, sample_rate=0.5)

# Train the random forest on the coarsened data
forest_model.fit(X_train, y_train, X_test, y_test)

# Make predictions on the train data
train_predictions = forest_model.predict(X_train, regression_method='mean')
print(f"mean train RSE: ", np.mean((train_predictions - y_train) ** 2) / np.var(y_train))

# Make predictions on the test data
test_predictions = forest_model.predict(X_test, regression_method='mean')
print(f"mean test RSE: ", np.mean((test_predictions - y_test) ** 2) / np.var(y_test))

Tree 1/10, Training RMSE (relative to variance): 0.36737751196707297
Tree 1/10, Testing RMSE (relative to variance): 2.1606816256650427
Tree 2/10, Training RMSE (relative to variance): 0.2556605625565928
Tree 2/10, Testing RMSE (relative to variance): 1.9535454430496595
Tree 3/10, Training RMSE (relative to variance): 0.16891020225723588
Tree 3/10, Testing RMSE (relative to variance): 2.1934854095494574
Tree 4/10, Training RMSE (relative to variance): 0.11434125669629085
Tree 4/10, Testing RMSE (relative to variance): 2.1693100283462
Tree 5/10, Training RMSE (relative to variance): 0.4106056187551459
Tree 5/10, Testing RMSE (relative to variance): 1.7467802335775215
Tree 6/10, Training RMSE (relative to variance): 0.2774281116971504
Tree 6/10, Testing RMSE (relative to variance): 2.445315523143284
Tree 7/10, Training RMSE (relative to variance): 0.22113873101278286
Tree 7/10, Testing RMSE (relative to variance): 2.3154315791474316
Tree 8/10, Training RMSE (relative to variance): 0.2851